In [ ]:
# validation set
# devide the data set into smaller sets for validation process

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
 data, target, test_size=0.2, random_state=42)


# 20%를 validation set으로
sub_input, val_input, sub_target, val_target = train_test_split(
 train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [2]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [5]:
# croos validation
# in order to achieve stable validation result and use more training data

from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)
print(type(scores))
# fit_time, score_time, test_score 키를 가진 딕셔너리를 반환
# fit_time: 각 모델 훈련 시간 / score_time: 검증하는 시간 / test_score
#  default 5-fold cross validation
import numpy as np
print(np.mean(scores['test_score']))



{'fit_time': array([0.02202153, 0.0153091 , 0.02137351, 0.02109694, 0.02716613]), 'score_time': array([0.00886273, 0.00793576, 0.00186968, 0.00679755, 0.00171709]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
<class 'dict'>
0.855300214703487


In [9]:
# 주의할 점은 cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않음
# train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비했기에 괜춘
#  만약 섞고 싶으면 splitter 지정



# cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용
# 분류모델일 경우에는 StratifiedKFold 사용
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))


# 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.855300214703487
0.8574181117533719


In [21]:
# hyperparameter tuning
# AutoML == 자동 하아퍼파라미터 튜닝
# 매개변수 tuning하는 과정에서, 최적의 parameter 찾기

from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
# 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장
print(gs.best_params_)
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 ‘mean_test_score’ 키에 저장되어 있
print(gs.cv_results_['mean_test_score'])

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])


0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [22]:
# 1. 먼저 탐색할 매개변수를 지정합니다
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
 'max_depth': range(5, 20, 1),
 'min_samples_split': range(2, 100, 10)
 }

# 2. 그다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾습니다.
# 이 조합은 그리드 서치 객체에 저장됩니다

# 3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련합니다.
# 이 모델도 그리드 서치 객체에 저장
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)

print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


In [25]:
# random search
# 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울
# 또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 때

from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(5)


params = {'min_impurity_decrease': uniform(0.0001, 0.001),
 'max_depth': randint(20, 50),
 'min_samples_split': randint(2, 25),
 'min_samples_leaf': randint(1, 25),
 }

from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_
print(dt.score(test_input, test_target))


{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
0.86
